Reference: https://github.com/YuandZhang/Dongchedi 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import csv
import requests # This command allows us to fetch URLs
from lxml import etree
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

In [2]:
class Config:
    stopwords_file = './hit_stopwords.txt'

In [3]:
from lxml import etree #etree是基于ElementTree API的一种XML解析方式
import unicodedata
import html
import re
import jieba
#from utils.Configuration import Config
import base64
def remove_html(content,isdecode=None):#isdecode 判断是否先需要解码,需要解码为1
    # 移除html标签
    # &#x等编码问题
    if isdecode==1:
        content=base64.b64decode(content).decode(encoding='utf-8')#内容解码
    content = html.unescape(content)
    content = unicodedata.normalize('NFKD', content)
    # 将html的换行替换成字符换行
    selector = etree.HTML(content)
    str_list = selector.xpath('//text()')
    text = ''.join(str_list)
    # text = clean(text)#数据清理
    text = clean(text)  # 去除英文和标点
    text=remove_stopwords(text)

    return text

def clean(text):
    text = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return text

def remove_stopwords(text):
    stopwords = [i.strip() for i in open(Config.stopwords_file,encoding='utf-8').readlines()]
    text_depart=jieba.cut(text.strip())
    outstr=''
    for word in text_depart:
        if word not in stopwords:
            if word !='\t':
                outstr+=word
                outstr+=" "
    return outstr

def seg_tail_split(str1, sep=r":|,|，|。|n|！|!|：|'\'"):  # 分隔符可为多样的正则表达式
    # 保留分割符号，置于句尾，比如标点符号
    wlist = re.split(sep, str1)
    seg_word = re.findall(sep, str1)
    seg_word.extend(" ")  # 末尾插入一个空字符串，以保持长度和切割成分相同
    wlist = [x + y for x, y in zip(wlist, seg_word)]  # 顺序可根据需求调换
    return wlist

def ListCharReplace(ls,o,n):
    #x:需要的列表
    #o:原来的字符
    #n:现在的字符
    b = list(ls[0])
    rep = [n if x == o else x for x in b]
    s =["".join(rep)]
    return s

In [4]:
#从csv里获取车型id和名称
def getcarid():
    caridls=[]
    carnamels=[]
    with open('data_dongchedi.csv',encoding='utf-8') as cf:
        lines=cf.readlines()
        for line in lines:
            carid, carname=line.split(',')[0],line.split(',')[1]
            caridls.append(carid)
            carnamels.append(carname)
        return caridls,carnamels

In [5]:
#获取车辆id对应的口碑网页链接
def getUrl(carid):
    #精华评价
    url = 'https://www.dongchedi.com/auto/series/score/{}-x-S0-x-x-x'.format(carid)
    return url

In [6]:
def getRoot(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36 Edg/86.0.622.43'}
    r=requests.get(url,headers=headers)
    # etree=html.etree
    root=etree.HTML(r.text)
    return root

In [7]:
#获取评论
def getComment(root,url,carid,carname):
    commentls=[]
    _url=url+'-1'
    root=getRoot(_url)        
    for comment in range(1,15): #默认每页只有14条数据，但是这个需要修改
        CommentRoot=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/article[{}]/section/p/text()'.format(str(comment))) 
        commentls.append(CommentRoot)
    saveComment(commentls,carid,carname,'csv')

In [85]:
#获取评论-加上翻页
def getComment(root,url,carid,carname):
    #获取页数
    pages=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/div[2]/ul/li[7]/a/span/text()')
    pages=int(pages[0])
    print(pages)
    
    commentls=[]

    for p in range(1,pages+1):
        _url=url+'-{}'.format(str(p))
        root=getRoot(_url)        

        for comment in range(1,15): #默认每页只有14条数据，但是这个需要修改
            CommentRoot=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/article[{}]/section/p/text()'.format(str(comment))) 
            commentls.append(CommentRoot)
    return commentls
    saveComment(commentls,carid,carname,'csv')
    
#把评论内容保存到csv中
def saveComment(commentls,carid,carname,filetype):
    if filetype=='csv':
        #需要删除所有空白值评论
        #while '' in str_list:
            #str_list.remove('')
        
        #把评论分行写入csv中
        for i in range(len(commentls)):
            with open('data_car_comment.csv','a',encoding='utf-8-sig',newline='') as dc:
                #rep_comment=ListCharReplace(commentls[i],'{','(')
                dcwriter = csv.writer(dc)
                dcwriter.writerow([carid,carname,commentls[i]])
                dc.close()
    elif filetype=='txt':
        for i in range(0, len(commentls)):
            with open('data_carcomment.txt', mode='a', encoding='utf-8', newline='') as dc:
                dc.writelines(carid+'||||'+carname+'||||'+commentls[i])

In [84]:
comment_text

[['理想，仿佛车市的一个爆点\n只要一说到“理想”二字\n就会带来前所未有的空前热度与热搜话题！\n一句“500W内最好的家用旗舰SUV”\n已经让它被捧上微博热搜第一名的头把交椅\n一众车评人同一时间铺天盖地的评测首发视频\n抖音头部网红抢破了头的争相争抢流量\n可以见得理想L9，未出世、以露锋芒…\n\n外观：APP上盲选的灰色车漆，没想到竟然成了之后大众选择的TOP 1车漆颜色，当看到实车的第一眼，便深深爱上那足足两米长的星环大灯（据说单单这个星环灯就是理想联合强大供应商海拉一起斥资两亿共同研发打造，视觉效果非常出色，灯带纯净，即使凑近看也没有颗粒感，中间无任何断点，也算是业内首创）仿佛外太空飞行器一般，未来感满满，直接击中我的心房🫀 就像初恋般的那微微一笑是如此的迷人，让我沉醉其中…\n\n\n车身尺寸：车身尺寸方面，L9 因为定位三代同堂、全家出行的关系，在尺寸上相比理想ONE来说又进化了不少。长宽高 分别来到了5,218 x 1,998 x 1,800 mm，轴距 3,105 mm，车身长度已经与与奔驰 GLS 持平，轴距则与宝马 X7 相当比，可谓是空间与操控的集大成者，对于全尺寸SUV来说宽度对地下车库相对比较友好，1800mm的高度对于绝大多数2米限高的地库高度来说也不会有不必要的刮顶困扰。而5米2的绝对长度给三排的乘坐空间以及舒适度带来前所未有的全新体验，堪称汽车界的“空间大师”\n\n\n内饰：中控、门板、扶手、座椅，只要是你手能够触及的地方均用了细腻的皮质包裹，同样这么大面积的皮质运用，在一台40万的车上真的是非常罕见，完全可以用前无古人来形容，毫不夸张！整体质感的打造上也可谓是毫不含糊可以说是同级无竞品，甚至可以越级去跟很多百万豪车掰掰手腕！\n\n\n配置：配置层面那就更不用多言了，厂长一句“标配即顶配”，直接宣告封顶。什么彩电、冰箱、大沙发全部都给你拿捏了。你想得到的它全有，你想不到的它还有，直接把180平米的大平层都给你搬车里来了，试问还能有啥可挑剔的呢？\n\n\n智能化：智能座舱可谓理想 L9 最大的亮点，在 L9 上理想目标把视听和交互体验拉到一个全新的高度，理想可谓在这智能座舱内加入了更多的极具前瞻性的创新点，率先开启了五屏交互时代，让理想 L9 从ONE原本基础上的“好开好用”演变到现在的“科技、奢华、越极、奢享”。而这些

In [80]:
caridls,carnamels=getcarid()
carid=caridls[1]
carname=carnamels[1]#当前爬取的车型
url = getUrl(carid)
root = getRoot(url)
comment_text = getComment(root, url,carid,carname)

6


In [87]:
saveComment(comment_text,carid,carname,"csv")

In [77]:
# 把长文本切成短句，目前还在报错，待修正

def TidyComment():
    originfile='data_car_comment.csv'
    tidyfile='data_car_comment_cleaned.csv'
    textid=1
    with open(originfile, mode='r', encoding='utf-8', newline='')as ofile:
        with open(tidyfile,mode='a',encoding='utf-8')as tfile:
            lines=ofile.readlines()
            for line in lines:
                sen_ls=[]
                senid=1
                carid=line.split('')[0]
                carname=line.split('')[1]
                carcomment=line.split('')[2][1:-2]
#分割文本-去停用词-

                split_comment_ls=seg_tail_split(carcomment)#分割成句子列表

                print('正在写入{}'.format(str(textid)))
                for sen in split_comment_ls:
                    sen=remove_stopwords(sen)
                    if len(sen)!=0:
                        sen_ls.append(sen)#生成新的句子列表sen_ls
                sen_len=len(sen_ls)#子句列表长度
                tfile.writelines(str(textid) + str(sen_len)+'\n')  # 写入文本序号,子句个数
                tfile.writelines(carid + ',' + carname + '\n')  # 写入车ID，车辆名称
                for _sen in sen_ls:
                    tfile.writelines(str(senid)+','+_sen+'\n')
                    senid=senid+1
                textid=textid+1

In [78]:
if __name__ == '__main__':
    # Spider()
    TidyComment()

ValueError: empty separator